# 03 — Data Harmonisation and Merge

This notebook merges cleaned WDI and WGI datasets into a
single cross-country panel dataset used for analysis.

## Inputs
- Clean WDI indicators
- Clean WGI governance indicators

## Output
`master_dataset.csv`

In [1]:
# =========================
# SETUP
# =========================

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
BASE_PATH = "/content/drive/MyDrive/thesis_project/"

INTERIM_DATA = BASE_PATH + "data/interim/"
PROCESSED_DATA = BASE_PATH + "data/processed/"

In [3]:
import pandas as pd

## Load Cleaned Indicator Datasets

In [4]:
oil = pd.read_csv(INTERIM_DATA + "wdi_oil_rents_clean.csv")
gdp_growth = pd.read_csv(INTERIM_DATA + "wdi_gdp_growth_clean.csv")
gdp_pc = pd.read_csv(INTERIM_DATA + "wdi_gdp_per_capita_clean.csv")
inflation = pd.read_csv(INTERIM_DATA + "wdi_inflation_clean.csv")
debt = pd.read_csv(INTERIM_DATA + "wdi_debt_clean.csv")
wgi = pd.read_csv(INTERIM_DATA + "wgi_governance_clean.csv")

## Create Country Lookup Table
Ensures country names remain consistent after merging.

In [5]:
country_lookup = pd.concat([
    oil[["Country Code","Country Name"]],
    gdp_growth[["Country Code","Country Name"]],
    gdp_pc[["Country Code","Country Name"]],
    inflation[["Country Code","Country Name"]],
    debt[["Country Code","Country Name"]],
    wgi[["Country Code","Country Name"]],
]).drop_duplicates()

## Remove Duplicate Country Name Columns

In [6]:
gdp_growth = gdp_growth.drop(columns=["Country Name"])
gdp_pc = gdp_pc.drop(columns=["Country Name"])
inflation = inflation.drop(columns=["Country Name"])
debt = debt.drop(columns=["Country Name"])
wgi = wgi.drop(columns=["Country Name"])

## Inspect Dataset Structures

In [7]:
for name, df in {
    "oil": oil,
    "gdp_growth": gdp_growth,
    "gdp_pc": gdp_pc,
    "inflation": inflation,
    "debt": debt,
    "wgi": wgi
}.items():
    print("\n", name)
    print(df.columns)


 oil
Index(['Country Name', 'Country Code', 'Year', 'Oil_Rents_GDP',
       'log_oil_rents'],
      dtype='object')

 gdp_growth
Index(['Country Code', 'Year', 'GDP_growth'], dtype='object')

 gdp_pc
Index(['Country Code', 'Year', 'GDP_per_capita', 'log_gdp_per_capita'], dtype='object')

 inflation
Index(['Country Code', 'Year', 'Inflation', 'Inflation_w'], dtype='object')

 debt
Index(['Country Code', 'Year', 'Gov_Debt_GDP', 'log_debt'], dtype='object')

 wgi
Index(['Country Code', 'Year', 'Control of Corruption',
       'Government Effectiveness', 'Rule of Law'],
      dtype='object')


## Sequential Dataset Merge
Outer joins preserve all country-year observations.

In [8]:
master = oil.copy()

In [9]:
master = oil.merge(
    gdp_growth,
    on=["Country Code","Year"],
    how="outer"
)
print(master.shape)

(11312, 6)


In [10]:
master = master.merge(
    gdp_pc,
    on=["Country Code","Year"],
    how="outer"
)
print(master.shape)

(11482, 8)


In [11]:
master = master.merge(
    inflation,
    on=["Country Code","Year"],
    how="outer"
)
print(master.shape)

(11638, 10)


In [12]:
master = master.merge(
    debt,
    on=["Country Code","Year"],
    how="outer"
)
print(master.shape)

(11638, 12)


In [13]:
master = master.merge(
    wgi,
    on=["Country Code","Year"],
    how="outer"
)
print(master.shape)

(11948, 15)


## Restore Country Names

In [14]:
master = master.merge(
    country_lookup,
    on="Country Code",
    how="left",
    suffixes=("", "_lookup")
)

master["Country Name"] = master["Country Name"].fillna(
    master["Country Name_lookup"]
)

master.drop(columns=["Country Name_lookup"], inplace=True)

In [15]:
master["Country Name"] = (
    master.groupby("Country Code")["Country Name"]
    .transform(lambda x: x.dropna().iloc[0] if x.notna().any() else None)
)

## Dataset Diagnostics

In [16]:
print("Countries:", master["Country Code"].nunique())
print("Years:", master["Year"].nunique())
print("Observations:", len(master))

Countries: 224
Years: 65
Observations: 12263


## Save Master Panel Dataset

In [17]:
master.to_csv(
    PROCESSED_DATA + "master_dataset.csv",
    index=False,
    float_format="%.10f"
)

## Pipeline Completion

In [18]:
print("✅ Master dataset created successfully.")

✅ Master dataset created successfully.
